In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import json

import sys
import os

sys.path.append(os.path.abspath('../'))
from _0_utils.tool import *

data_root = '../../MyData/'

## Load data

In [8]:
item1_embedding = pd.read_csv(data_root+'merged_1197.csv')
item1_embedding.columns

Index(['cik', 'Year', 'item1', 'item1_word_count', 'item1_token_count',
       'item1_summary', 'Error', 'ErrorMessage', 'item1_summary_token_count',
       'item1_summary_word_count', 'item1_embeddings', 'tic', 'Unnamed: 0',
       'conm', 'gvkey', 'SIC_Division', 'SIC_MajorGroup', 'NAICS_Sector',
       'NAICS_SubSector', 'hoberg_fic25', 'hoberg_fic100', 'GICS_Sector',
       'GICS_IndustryGroup', 'GICS_Industry', 'cusip', 'SP_ENTITY_NAME',
       'SP_ENTITY_ID', 'SP_SHORT_DESC', 'SP_LONG_DESC', 'SP_TOPICTAG',
       'SP_CATEGORIES', 'SP_CUSIP', 'SP_TICKER', 'SP_EXCHANGE_TICKER',
       'SP_COMPANY_NAME', '_merge', 'word_count', 'YH_BUS_DESC', 'SA_BUS_DESC',
       'ORBIS_PROD_SERV', 'ORBIS_OVERVIEW'],
      dtype='object')

In [44]:
item1_embedding['SIC_Division'].value_counts()

SIC_Division
D    480
H    212
I    164
E    125
G     86
B     49
F     44
C     26
K      3
Name: count, dtype: int64

In [9]:
other_embedding = pd.read_csv(data_root+'output_embeddings_2.csv')
other_embedding.columns

Index(['Unnamed: 0', 'cik', 'Year', 'conm', 'SP_SHORT_DESC', 'SP_LONG_DESC',
       'ORBIS_PROD_SERV', 'ORBIS_OVERVIEW', 'SP_SHORT_DESC_embeddings',
       'SP_LONG_DESC_embeddings', 'ORBIS_PROD_SERV_embeddings',
       'ORBIS_OVERVIEW_embeddings'],
      dtype='object')

In [3]:
item1_embedding = pd.read_csv(data_root+'merged_1197.csv')
item1_embedding = item1_embedding[['cik', 'tic', 'Year', 'item1_embeddings']]
print(len(item1_embedding))
print(item1_embedding.head())

other_embedding = pd.read_csv(data_root+'output_embeddings_2.csv')
other_embedding = other_embedding[['cik', 'SP_SHORT_DESC_embeddings', 'SP_LONG_DESC_embeddings', 'ORBIS_PROD_SERV_embeddings', 'ORBIS_OVERVIEW_embeddings']]
print(len(other_embedding))
print(other_embedding.head())

total_embedding = pd.merge(item1_embedding, other_embedding, on=['cik'])

# To reduce mem consumption
item1_embedding = ''
other_embedding = ''

print(len(total_embedding))
total_embedding.head()

1197
       cik   tic  Year                                   item1_embeddings
0   850460  WIRE  2021  [0.03955410048365593, -0.04159577935934067, -0...
1   352541   LNT  2021  [-0.0231856107711792, 0.001279839314520359, 0....
2  1704715   AMR  2021  [0.014074714854359627, 0.006938479840755463, 0...
3  1575515   SFM  2021  [-0.023708730936050415, 0.01762891560792923, 0...
4  1125376  ENSG  2021  [0.04463111609220505, 0.0018805989529937506, 0...
1197
       cik                           SP_SHORT_DESC_embeddings  \
0   850460  [0.01568225771188736, -0.07636360824108124, -0...   
1   352541  [-0.013836896046996117, -0.028995024040341377,...   
2  1704715  [0.0030535957776010036, 0.0008783274097368121,...   
3  1575515  [-0.025762900710105896, 0.0034106436651200056,...   
4  1125376  [0.04616139456629753, -0.0021259395871311426, ...   

                             SP_LONG_DESC_embeddings  \
0  [0.06089901179075241, -0.07059630751609802, -0...   
1  [-0.021961161866784096, 0.02030270546674

,cik,tic,Year,item1_embeddings,SP_SHORT_DESC_embeddings,SP_LONG_DESC_embeddings,ORBIS_PROD_SERV_embeddings,ORBIS_OVERVIEW_embeddings
0,850460,WIRE,2021,"[0.03955410048365593, -0.04159577935934067, -0...","[0.01568225771188736, -0.07636360824108124, -0...","[0.06089901179075241, -0.07059630751609802, -0...",NaN,NaN
1,352541,LNT,2021,"[-0.0231856107711792, 0.001279839314520359, 0....","[-0.013836896046996117, -0.028995024040341377,...","[-0.021961161866784096, 0.020302705466747284, ...","[-0.015400929376482964, 0.03232719004154205, -...","[-0.041709959506988525, -0.011898815631866455,..."
2,1704715,AMR,2021,"[0.014074714854359627, 0.006938479840755463, 0...","[0.0030535957776010036, 0.0008783274097368121,...","[-0.0017570963827893138, 0.006557094398885965,...","[-0.02030838653445244, -0.009386725723743439, ...","[-0.012340557761490345, -0.01428779773414135, ..."
3,1575515,SFM,2021,"[-0.023708730936050415, 0.01762891560792923, 0...","[-0.025762900710105896, 0.0034106436651200056,...","[-0.015190708450973034, 0.011541897431015968, ...","[-0.013177500106394291, 0.036401789635419846, ...","[-0.01883152313530445, 0.021741388365626335, 0..."
4,1125376,ENSG,2021,"[0.04463111609220505, 0.0018805989529937506, 0...","[0.04616139456629753, -0.0021259395871311426, ...","[0.05935207009315491, 0.0013673527864739299, 0...","[0.029405493289232254, 0.014010551385581493, 0...","[-0.006068837363272905, 0.010537531226873398, ..."


### Check dataset

In [4]:
nan_proportion = total_embedding.isna().mean()
print(f"The nan value proportion in each column:\n{nan_proportion}\n\n")

The nan value proportion in each column:
cik                           0.000000
tic                           0.000000
Year                          0.000000
item1_embeddings              0.000000
SP_SHORT_DESC_embeddings      0.041771
SP_LONG_DESC_embeddings       0.095238
ORBIS_PROD_SERV_embeddings    0.208020
ORBIS_OVERVIEW_embeddings     0.168755
dtype: float64




## Obtain tensors and prepare data

In [5]:
target_list = ['item1_embeddings', 'SP_LONG_DESC_embeddings', 'ORBIS_OVERVIEW_embeddings']
info_list = ['cik', 'tic', 'Year']

exp_df = convert_to_array(total_embedding, info_list, target_list, 1536, False)
print(type(exp_df))
print(exp_df.shape)


item1_np = np.vstack(exp_df['item1_embeddings'].values)
sp_long_np = np.vstack(exp_df['SP_LONG_DESC_embeddings'].values)
orbis_np = np.vstack(exp_df['ORBIS_OVERVIEW_embeddings'].values)

d:\NGNE\clustering_project\CP3106\_0_utils\tool.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[target] = data_df[target].apply(


<class 'pandas.core.frame.DataFrame'>
(1197, 6)


In [6]:
sp_long_np = sp_long_np[~np.all(np.isnan(sp_long_np), axis=1)]
orbis_np = orbis_np[~np.all(np.isnan(orbis_np), axis=1)]
X = np.vstack([item1_np, sp_long_np, orbis_np])
print(X.shape)

(3275, 1536)


In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset, random_split

# Convert NumPy array to PyTorch tensor
X_tensor = torch.tensor(X, dtype=torch.float32)

# Custom Dataset class for VAE (no labels)
class AE_Dataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Return only the input data (no labels)
        return self.features[idx]

# Create the full dataset
dataset = AE_Dataset(X_tensor)

# Split dataset into train and test (80% train, 20% test)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for train and test
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Check the shape of a batch
for batch in train_loader:
    print(batch.shape)  # (32, 10) assuming batch size of 32
    break  # Just print the first batch


torch.Size([64, 1536])
